<a href="https://colab.research.google.com/github/Bonorinoa/MapReduce_Python/blob/main/MapReduce_TweetSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -f hadoop-3.3.2.tar.gz*

In [2]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz

--2022-04-29 14:06:07--  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638660563 (609M) [application/x-gzip]
Saving to: ‘hadoop-3.3.2.tar.gz’

hadoop-3.3.2.tar.gz 100%[===================>] 609.07M  19.1MB/s    in 34s     

2022-04-29 14:06:41 (18.1 MB/s) - ‘hadoop-3.3.2.tar.gz’ saved [638660563/638660563]



In [3]:
!tar -xzf hadoop-3.3.2.tar.gz

In [4]:
#copy  hadoop file to user/local
!cp -r hadoop-3.3.2/ /usr/local/

In [5]:
#To find the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


To set the java path, go to /usr/local/hadoop-3.3.2/etc/hadoop/hadoop-env.sh then

* Look for the line `export JAVA_HOME=`
* Replace it with `export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/`
* Remove the comments (the notebook will automatically save the shell script)

In [6]:
# Chek if Hadoop was correctly installed
!/usr/local/hadoop-3.3.2/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

In [7]:
!pip install mrjob

     |████████████████████████████████| 439 kB 7.6 MB/s 


## The goal is to compute the sentiment score of a given tweet.
### Problem:



In [8]:
!pip install transformers --quiet

     |████████████████████████████████| 4.0 MB 8.5 MB/s 
     |████████████████████████████████| 596 kB 66.7 MB/s 
     |████████████████████████████████| 895 kB 48.7 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 


In [9]:
# Load Groceries.csv or tinyGroceries.csv
from google.colab import files
import pandas as pd

uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [10]:
tweets = pd.read_csv("Tweets.csv")
tweets.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [22]:
# mini dataset for testing
# Feel free to subset as many tweets as you like

tinyTweets = tweets['text'][:50]
tinyTweets.to_csv("tinyTweets.csv")

## MRJob implementation of above python program

In [94]:
%%file tweetsSentiment.py
#!/usr/bin/env python

from mrjob.job import MRJob
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

model_name = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

class tweetSentiment(MRJob):
  

    def mapper(self, _, line):

      cols = line.split(',')

      word_tokens = word_tokenize(str(cols[1]))

      filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

      filtered_sentence = []

      for w in word_tokens:

        if w not in stop_words:
        
          filtered_sentence.append(w)

      yield "".join(filtered_sentence), 1


    def combiner(self, word, counts):

      results = classifier(word)
      yield results[0]['label'], 1
     

    def reducer(self, word, counts):

      yield word, sum(counts)


if __name__ == '__main__':
    tweetSentiment.run()

Overwriting tweetsSentiment.py


In [98]:
# Change below the name of the Python/MrJob program and the file(s) used by it
# The datafiles can have extensions (e.g. .csv, .txt)  or be without extension.
!python tweetsSentiment.py tinyTweets.csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using configs in /root/.mrjob.conf
No configs specified

## Let's run it in Hadoop now (has some issues)

In [90]:
# %%file ~/.mrjob.conf
# runners:
#     hadoop:
#       hadoop_bin: /usr/local/hadoop-3.3.2/bin/hadoop
#       hadoop_streaming_jar: /usr/local/hadoop-3.3.2/share/hadoop/tools/lib/hadoop-streaming-3.3.2.jar
#       hadoop_tmp_dir: file:///content/tmp/mrjob

Overwriting /root/.mrjob.conf


In [96]:
# Change below the name of the Python/MrJob program and the file(s) used by it
# Note that the option -r hadoop changes the execution  from local to the hadoop framework
# !python tweetsSentiment.py -r hadoop tinyTweets.csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using configs in /root/.mrjob.conf
Using Hadoop version